In [6]:
# Import necessary libraries
import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pymp  # For parallel processing
import time  # For timing
import math

In [7]:
print(hasattr(pymp, 'Parallel'))

True


In [3]:
dir(pymp)

['Parallel',
 '_IterableQueueIterator',
 '_LOGGER',
 '_Queue',
 '_QueueIterator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_config',
 '_functools',
 '_is_py2',
 '_logging',
 '_multiprocessing',
 '_os',
 '_platform',
 '_shared',
 '_sys',
 '_time',
 'config',
 'print_function',
 'shared']

In [8]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [10]:
input_folder = '/Users/vishvajith/Desktop/COA project/sobel_input/Animals'
serial_output_folder = '/Users/vishvajith/Desktop/COA project/sobel_output/Serial_out'
parallel_output_folder = '/Users/vishvajith/Desktop/COA project/sobel_output/Parallel_out'

In [11]:
# Create dropdown widget for selecting the processing method
compute_mode_dropdown = widgets.Dropdown(
    options=['Serial', 'Parallel'],
    description='Compute Mode:',
)

# Display the dropdown widget
display(compute_mode_dropdown)

Dropdown(description='Compute Mode:', options=('Serial', 'Parallel'), value='Serial')

In [13]:
# Create a button to start the processing
process_button = widgets.Button(description='Process Images')


Running Serial Processing...

Serial processing time: 7.5011 seconds

Running Parallel Processing...

Parallel processing time: 3.1513 seconds

Timing Comparison:
Serial processing time: 7.5011 seconds
Parallel processing time: 3.1513 seconds
Parallel processing was faster by 4.3498 seconds


In [15]:
# Function to process images serially
def process_images_serial():
    # Create the output folder if it doesn't exist
    if not os.path.exists(serial_output_folder):
        os.makedirs(serial_output_folder)
        
    # Get list of all image files in the input folder
    image_files = os.listdir(input_folder)
    
    # Measure start time
    start_time = time.time()
    
    for image_file in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(input_folder, image_file)
        
        # Read the image
        image = cv2.imread(image_path)
        
        # Check if the image was successfully loaded
        if image is None:
            print(f"Failed to read image: {image_path}")
            continue
        
        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply the Sobel operator
        sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
        sobel_combined = cv2.magnitude(sobelx, sobely)
        sobel_combined = cv2.convertScaleAbs(sobel_combined)
        
        # Save the processed image to the output folder
        output_image_path = os.path.join(serial_output_folder, 'serial_' + image_file)
        cv2.imwrite(output_image_path, sobel_combined)
    
    # Measure end time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nSerial processing time: {total_time:.4f} seconds")
    return total_time




In [20]:
# Cell 6: Function to process images in parallel using pymp
def process_images_parallel():
    # Create the output folder if it doesn't exist
    if not os.path.exists(parallel_output_folder):
        os.makedirs(parallel_output_folder)
        
    # Get list of all image files in the input folder
    image_files = os.listdir(input_folder)
    
    # Measure start time
    start_time = time.time()
    
    # Use pymp for parallel processing
    num_threads = 4  # You can adjust this number based on your system
    shared_image_files = pymp.shared.array((len(image_files),), dtype='S100')
    for idx, file in enumerate(image_files):
        shared_image_files[idx] = file.encode('utf-8')
    
    with pymp.Parallel(num_threads) as p:
        for i in p.range(len(shared_image_files)):
            image_file = shared_image_files[i].decode('utf-8')
            # Construct the full path to the image file
            image_path = os.path.join(input_folder, image_file)
            
            # Read the image
            image = cv2.imread(image_path)
            
            # Check if the image was successfully loaded
            if image is None:
                print(f"Failed to read image: {image_path}")
                continue
            
            # Convert the image to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Apply the Sobel operator
            sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
            sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
            sobel_combined = cv2.magnitude(sobelx, sobely)
            sobel_combined = cv2.convertScaleAbs(sobel_combined)
            
            # Save the processed image to the output folder
            output_image_path = os.path.join(parallel_output_folder, 'parallel_' + image_file)
            cv2.imwrite(output_image_path, sobel_combined)
    
    # Measure end time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nParallel processing time: {total_time:.4f} seconds")
    return total_time




In [21]:
# Define the function that runs when the button is clicked
def on_process_button_clicked(b):
    compute_mode = compute_mode_dropdown.value
    
    if compute_mode == 'Serial':
        print("\nRunning Serial Processing...")
        serial_time = process_images_serial()
        print("\nRunning Parallel Processing...")
        parallel_time = process_images_parallel()
    elif compute_mode == 'Parallel':
        print("\nRunning Parallel Processing...")
        parallel_time = process_images_parallel()
        print("\nRunning Serial Processing...")
        serial_time = process_images_serial()
    else:
        print("Invalid Compute Mode Selected.")
        return
    # Print comparison
    print(f"\nTiming Comparison:")
    print(f"Serial processing time: {serial_time:.4f} seconds")
    print(f"Parallel processing time: {parallel_time:.4f} seconds")
    time_diff = serial_time - parallel_time
    if time_diff > 0:
        print(f"Parallel processing was faster by {time_diff:.4f} seconds")
    else:
        print(f"Serial processing was faster by {-time_diff:.4f} seconds")

In [22]:
# Cell 8: Link the button click event to the function
process_button.on_click(on_process_button_clicked)

# Display the process button
display(process_button)

Button(description='Process Images', style=ButtonStyle())